In [1]:
# Importing requried modules for this analysis
import os
import csv
import pandas as pd

In [2]:
# Fields required for this analysis
fields = ['Created Date', 'Complaint Type', 'Incident Zip', 'Borough']

# Read 311 service request data file and assign to dataframe df using only the required fiels
df=pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', sep=',', dtype='object', usecols=fields)

# Rename service request dataframe columns
df.columns = ['year', 'complaintType', 'zip', 'borough']

# Stage the service request dataframe to skip this step in future runs using pickle in next cell
# sr_all -> service request all records; no records dropped
df.to_pickle('sr_all.pkl')

In [3]:
# Read service request all records dataframe if the pickle exists
if os.path.exists('sr_all.pkl'): 
    df=pd.read_pickle('sr_all.pkl')

In [4]:
# Extract year from Create Date field (renamed to year in earlier cell)
df['year'] =  pd.to_datetime(df['year']).apply(lambda x: x.year)

# Stage the service request dataframe to skip this step in future runs using pickle in next cell
# sr_year_all -> service request year extracted all records
df.to_pickle('sr_year_all.pkl')

In [5]:
# Read service request dataframe if the pickle exists
if os.path.exists('sr_year_all.pkl'): 
    df = pd.read_pickle('sr_year_all.pkl')

In [6]:
# validate extracted year values
df.year.unique().tolist()

[2016, 2011, 2015, 2010, 2014, 2017, 2013, 2012, 2018]

In [7]:
# Read zip code population and store in dictionary {zip:population}
zip_dict = {}
with open ('2010_Census_Populatio_By_Zipcode.csv') as infile:
    next(infile)
    reader = csv.reader(infile)
    for row in reader:
        zip_dict[row[0]] = row[1]

In [8]:
# Validate zip codes and population if all digits and zip codes are 5-digit long
[(key, value) for key, value in zip_dict.iteritems() \
                                        if not (key.isdigit() and \
                                                value.isdigit()) \
                                            or len(key)!=5]

[]

In [9]:
# Assign the population of zip codes to pop column
df['pop'] = df['zip'].map(zip_dict)

# Drop records with no population as it is basis for this analysis
df = df.dropna(subset=['pop'])

# Stage the service request dataframe to skip this step in future runs using picke in next cell
# sr_pop -> service request population populated and records with invalid records dropped
df.to_pickle('sr_pop.pkl')

In [10]:
# Read service request dataframe if the pickle exists
if os.path.exists('sr_pop.pkl'): 
    df = pd.read_pickle('sr_pop.pkl')

In [11]:
# validate that no records have missin population
len(df) - df['pop'].count()

0

In [12]:
# Read NYC boroughs' zip codes population in dictionary {zip:borough}
# bz_dict -> borough zip dictionary
bz_dict={}
with open ('borough_zip.txt') as infile: 
    reader = csv.reader(infile)
    for row in reader:
        for i in range(1, len(row)):
            bz_dict[row[i].strip()]=row[0].strip().upper()

In [13]:
# Validating zip codes in service request dataframe 
#   by checking the zip code exist in NYC zip code and discard the rest of records
df=df[df['zip'].isin(bz_dict)]

# Assign boroughs from NYC borough zip dictionary to updated_borough new column in service request dataframe
df['updated_borough'] = df['zip'].map(bz_dict)

# Drop service request records where borough in original data is different 
#   from that of extracted from NYC borough zip dictionary
#   The assumption here is that zip codes in service request data are valid compared to boroughs
#   since also populations that are required for this analysis are derived from zip codes
df=df[df['updated_borough']==df['borough']]

# Removing updated_borough column since it was just used for validating boroughs
df=df[['year','complaintType','zip','borough','pop']]

# Converting population to integer for calculations in this analysis
df['pop']=df['pop'].astype(int)

# Stage the service request dataframe to skip this step in future runs using pickle in next cell
# sr_final -> service request dataframe ready to be used for analysis
df.to_pickle('sr_final.pkl')

In [14]:
# Read service request dataframe if the pickle exists
if os.path.exists('sr_final.pkl'): 
    df=pd.read_pickle('sr_final.pkl')

In [15]:
# creating subset of service request data pertaining to 2017 for the analysis
df_2017=df[df['year']==2017]

In [16]:
# Overall (across years) top 10 complaints dataframe
df_top10_complaint = df.groupby(['complaintType']).size().sort_values(ascending=False).head(10).reset_index()

# convert top 10 complaints to list
top10_complaint = df_top10_complaint['complaintType'].tolist()

# 2017 service request data where complaint in overall top 10 
df_2017_top10_complaint = df_2017[df_2017['complaintType'].isin(top10_complaint)]

# Count of top 10 complaint type in 2017
df_top10_complaint_in2017_by_borough = df_2017_top10_complaint.groupby(['borough','complaintType']).size().reset_index(name='count2017')

# Sort by borough and complaint type
df_top10_complaint_in2017_by_borough.sort_values(by=['borough', 'complaintType'])

,borough,complaintType,count2017
0,BRONX,Blocked Driveway,24573
1,BRONX,HEAT/HOT WATER,68713
2,BRONX,Illegal Parking,16122
3,BRONX,Noise,3056
4,BRONX,Noise - Residential,57663
5,BRONX,PLUMBING,16488
6,BRONX,Street Condition,11155
7,BRONX,Street Light Condition,8410
8,BRONX,Water System,9971
9,BROOKLYN,Blocked Driveway,48866


In [22]:
# 10 most populous zip codes
df_top10_zip = df.groupby(['zip','pop']).size().reset_index().sort_values(by='pop',ascending=False).head(10)

# Convert 10 most populous zip codes to list
top10_zip = df_top10_zip['zip'].tolist()

# 2017 service request data where zip code population in top 10 
df_2017_top10_zip = df_2017_top10_complaint[df_2017_top10_complaint['zip'].isin(top10_zip)]

# Count of top 10 complaint type in 2017 for 10 most populous zip codes
df_top10_complaint_in2017_by_topZip = df_2017_top10_zip.groupby(['complaintType']).size().reset_index(name='count2017')

# Count of overall to 10 complaint types in 2017 for the most populous zip codes 
df_top10_complaint_in2017_by_topZip

,complaintType,count2017
0,Blocked Driveway,23841
1,HEAT/HOT WATER,29853
2,Illegal Parking,16515
3,Noise,3992
4,Noise - Residential,28944
5,PLUMBING,7350
6,Street Condition,7880
7,Street Light Condition,4498
8,Water System,6430


In [23]:
# Get the borough zip code population (population for each borough, zip)
df_borough_pop=df.groupby(['borough','zip']).first().reset_index()

# Calculate borough population by summing population for each borough
df_borough_pop=df_borough_pop.groupby('borough')['pop'].agg('sum').reset_index()

# Normalizing population (deviding boroughs' population by total sum)
df_borough_pop['popNormalized']  = df_borough_pop['pop'].div(df_borough_pop['pop'].sum())

# Sum and normalized population per borough
df_borough_pop

,borough,pop,popNormalized
0,BRONX,1382480,0.169566
1,BROOKLYN,2504700,0.307209
2,MANHATTAN,1565608,0.192027
3,QUEENS,2231553,0.273707
4,STATEN ISLAND,468730,0.057491


In [24]:
# Count of complaints for each borough
df_borough_complaint=df.groupby(['borough']).size().to_frame('count').reset_index()

# Normalizing count of complaints (dividing boroughs' complaint count by total sum)
df_borough_complaint['countNormalized']  = df_borough_complaint['count'].div(df_borough_complaint['count'].sum())

# Count and Normalized count of complaints
df_borough_complaint

,borough,count,countNormalized
0,BRONX,3138395,0.185736
1,BROOKLYN,5332984,0.315616
2,MANHATTAN,3431077,0.203057
3,QUEENS,4071462,0.240957
4,STATEN ISLAND,923155,0.054634


In [25]:
# Joining normalized population and complaint count to calculate complaint-index 
key='borough'
df_complaint_index=df_borough_complaint.join(df_borough_pop.set_index(key), on=key)

# keep subset of index dataframe columns 
df_complaint_index=df_complaint_index[['borough', 'countNormalized', 'popNormalized']]

# Calculating complaint-index
df_complaint_index['complaint-index']=df_complaint_index['countNormalized']/df_complaint_index['popNormalized']

# biggest complainers based on complaint-index
df_complaint_index

,borough,countNormalized,popNormalized,complaint-index
0,BRONX,0.185736,0.169566,1.095364
1,BROOKLYN,0.315616,0.307209,1.027364
2,MANHATTAN,0.203057,0.192027,1.057444
3,QUEENS,0.240957,0.273707,0.880345
4,STATEN ISLAND,0.054634,0.057491,0.950302
